# Functions


In [1]:
import skimage as ski
from google.colab import drive
import pandas as pd
from skimage import io
from IPython.display import clear_output
import matplotlib
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import tifffile as tiff
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
DONE_ASCII = r"""
 ____   ___  _   _  _____
|  _ \ / _ \| \ | || ____|
| | | | | | |  \| ||  _|
| |_| | |_| | |\  || |___
|____/ \___/|_| \_||_____|
"""


In [3]:
pip install tifffile

In [4]:
def normalize(img):
  img=(img-np.min(img))/(np.max(img)-np.min(img))
  return img
def normalize_255(img):
  img=((img-np.min(img))/(np.max(img)-np.min(img)))*255
  return img


In [5]:
%pip install readimc
from readimc import MCDFile, TXTFile

# Folder Creation for New Projects

In [7]:
bool_exist=False
while bool_exist==False:
 name_path = input("Enter the path where the project will be created (if the folder is in 'MyDrive' click on Enter): ")
 path = "/content/gdrive/MyDrive"
 path += "/"+name_path + "/"
 bool_exist=os.path.isdir(path)
 if bool_exist==False:
   print("❌ The path does not exist")
 else:
  bool_exist=True
project = input("Enter the project name: ")
if os.path.isdir(path+ project) == False:
    os.mkdir(path + project)
    print("✅ Project created")
else:
    print("✅ project already exist")
path=path+project+"/"
path_img=path+"images/"
path_mcd=path+"images/mcd/"
path_raw=path+"images/raw/"
if os.path.isdir(path_img)==False:
  os.mkdir(path_img)
  print("✅ Folder for images created")
if os.path.isdir(path_mcd)==False:
 os.mkdir(path_mcd)
 print("✅ Folder for MCD images created")
if os.path.isdir(path_raw)==False:
 os.mkdir(path_raw)
 print("✅ Folder for raw TIFF images created")


Enter the path where the project will be created (if the folder is in 'MyDrive' click on Enter): these/pipeline
Enter the project name: VAA
✅ project already exist


# Images format: MCD --> TIFF
Execute this block only if you have mcd files

***Put the MCD files in the folder "mcd" (project/images/mcd)***

### Functions

In [10]:
def convert_mcd_tiff(path_mcd,roi_exclude=[],marker_exclude=[],path="./images/raw_images"):
    if os.path.isdir(path)==False:
        os.mkdir(path)
    for file in os.listdir(path_mcd):
        print("   ** "+file+" **")
        with MCDFile(path_mcd+"/"+file) as f:
            slide = f.slides[0]
            panorama = slide.panoramas[0]
            for acq in range(len(slide.acquisitions)):
                acquisition = slide.acquisitions[acq]
                roi=acquisition.description
                if roi not in roi_exclude:
                    print("     ROI: "+roi)
                    if os.path.isdir(path+"/"+acquisition.description)==False:
                         os.mkdir(path+"/"+acquisition.description)
                    try:
                        img = f.read_acquisition(acquisition)
                        list_target=acquisition.channel_labels
                        dico_target={v:i for i,v in enumerate(list_target)}
                        for i in range(len(list_target)):
                            if list_target[i] not in marker_exclude:
                                img_marker=img[dico_target[list_target[i]],:,:]
                                cv2.imwrite(path+"/"+acquisition.description+"/"+list_target[i]+".tiff",img_marker)

                    except:
                        print("     Erreure: "+roi)
    print("✅ All the images are available in the folder: "+path)




### Execution

In [11]:
convert_mcd_tiff(path_mcd=path_mcd,path=path_raw)


   ** Lame_1.mcd **
     ROI: 19U07351a
     ROI: 19U07351b
     ROI: 19U07351c
     ROI: 19U07351d
     ROI: 19U07351e
     ROI: 19U07351f
     ROI: 19U07351g
     ROI: 20U01680a
     ROI: 20U01680b
     ROI: 20U01680c
     ROI: 20U01680d
     ROI: 20U01680e
     ROI: 20U04212a
     ROI: 20U04212b
     ROI: 20U04212c
     ROI: 20U04212d
     ROI: 20U04212e
     ROI: 20U04212f
     ROI: 20U06040a
     ROI: 20U06040b
     ROI: test
     ROI: test2
     ROI: test3
     ROI: test4
     ROI: 19U07351bb
     ROI: ROI_029
✅ All the images are available in the folder: /content/gdrive/MyDrive/these/pipeline/VAA//images/raw/


# Standart pre-processing
Image processing to enable better viewing
Threshold on the lower and upper quantil + arcsinh with cofactor + CLAHE (Contrast Limited Adaptive Histogram Equalization)

In [12]:
path_img=path+"/images/"
path_raw=path+"/images/raw/"
path_img_preprocessed1=path+"/images/img_processing_1/"
path_img_preprocessed1_marker=path_img_preprocessed1+"Marker/"
path_img_preprocessed1_biopsies=path_img_preprocessed1+"Biopsies/"

In [13]:
if os.path.isdir(path_img_preprocessed1)==False:
  os.mkdir(path_img_preprocessed1)
  print("✅ Folder for images preprocessed created")
if os.path.isdir(path_img_preprocessed1_biopsies)==False:
  os.mkdir(path_img_preprocessed1_biopsies)
  print("✅ Folder for images preprocessed created")
if os.path.isdir(path_img_preprocessed1_marker)==False:
  os.mkdir(path_img_preprocessed1_marker)

✅ Folder for images preprocessed created
✅ Folder for images preprocessed created


### Functions

In [14]:
import numpy as np
from skimage import io, exposure, util, filters, morphology

def imc_display_transform(
    img,               # 2D numpy array, IMC channel (float32/float64/uint16)
    low_q=1.0,         # lower quantile (background)
    high_q=99.9,       # upper quantile (clip)
    cofactor=5.0,      # arcsinh cofactor
    clahe=False,       # True to apply CLAHE (for visualization only)
    clahe_kernel=128,  # CLAHE tile size
    clahe_clip=0.01    # CLAHE clip limit
):
    x = img.astype(np.float32)

    # 1) Background subtraction (based on lower quantile)
    bg = np.quantile(x, low_q / 100.0)
    x = np.clip(x - bg, 0, None)

    # 2) Robust upper clipping
    hi = np.quantile(x, high_q / 100.0) if np.isfinite(x).all() else np.percentile(x, high_q)
    if hi > 0:
        x = np.clip(x, 0, hi)

    # 3) Arcsinh scaling (CyTOF-style)
    x = np.arcsinh(x / max(cofactor, 1e-6))

    # 4) Rescale to 0–1 range
    x = x - x.min()
    if x.max() > 0:
        x = x / x.max()

    # 5) Optional CLAHE (for display enhancement)
    # skimage CLAHE expects 8-bit or float [0,1]
    if clahe:
        x = exposure.equalize_adapthist(x, kernel_size=clahe_kernel, clip_limit=clahe_clip)

    return x  # float [0,1] ready for display or 8-bit conversion

# --- Example usage per channel ---
# raw = io.imread("path/my_channel.tiff")  # 16/32-bit
# disp = imc_display_transform(raw, low_q=1.0, high_q=99.9, cofactor=5.0, clahe=False)
# io.imsave("channel_display.png", util.img_as_ubyte(disp))


### Execution

In [15]:
low_q=float(input("Enter a low quantile (1.0 by default): "))
high_q=float(input("Enter a high quantile (99.99 by default): "))
cofactor=float(input("Enter a cofactor (5.0 by default): "))

Enter a low quantile (1.0 by default): 5
Enter a high quantile (99.9 by default): 99.999
Enter a cofactor (5.0 by default): 5


In [17]:
total_images = 0
for roi in os.listdir(path_raw):
    roi_dir = os.path.join(path_raw, roi)
    if not os.path.isdir(roi_dir):
        continue
    for marker in os.listdir(roi_dir):
        if os.path.isfile(os.path.join(roi_dir, marker)):
            total_images += 1

with tqdm(total=total_images, desc="IMC preprocessing", unit="img", ncols=100) as pbar:
    for roi in os.listdir(path_raw):

        roi_in = os.path.join(path_raw, roi)
        if not os.path.isdir(roi_in):
            continue

        out_biopsy = os.path.join(path_img_preprocessed1_biopsies, roi)
        os.makedirs(out_biopsy, exist_ok=True)

        for marker in os.listdir(roi_in):
            marker_in_path = os.path.join(roi_in, marker)
            if not os.path.isfile(marker_in_path):
                continue

            marker_name = marker[:-5]
            out_marker_dir = os.path.join(path_img_preprocessed1_marker, marker_name)
            os.makedirs(out_marker_dir, exist_ok=True)

            img = tiff.imread(marker_in_path)
            img_disp = imc_display_transform(img, low_q=low_q, high_q=high_q, cofactor=cofactor)

            cv2.imwrite(
                os.path.join(out_biopsy, marker_name + ".png"),
                normalize_255(img_disp)
            )
            cv2.imwrite(
                os.path.join(out_marker_dir, roi + ".png"),
                normalize_255(img_disp)
            )

            pbar.update(1)
print("\n ✅ Processed images ordered by biopsies available there: "+path_img_preprocessed1_biopsies)
print("✅ Processed images ordered by marker available there: "+path_img_preprocessed1_marker)
print(DONE_ASCII)

IMC preprocessing: 100%|█████████████████████████████████████████| 960/960 [02:45<00:00,  5.82img/s]


 ✅ Processed images ordered by biopsies available there: /content/gdrive/MyDrive/these/pipeline/VAA//images/img_processing_1/Biopsies/
✅ Processed images ordered by marker available there: /content/gdrive/MyDrive/these/pipeline/VAA//images/img_processing_1/Marker/

 ____   ___  _   _  _____
|  _ \ / _ \| \ | || ____|
| | | | | | |  \| ||  _|
| |_| | |_| | |\  || |___
|____/ \___/|_| \_||_____|



# Pipeline of Pre-processing
Generating pre-processed images with less noise ordered by biopsy
Pseudo code of the pre-processing:


*   Arcsinh_image=Arcsinh(image*cofactor) --> Increase variance between low and high pixel intensity
*   Blur_image=Blur(image) --> Remove noise between real cells

*   Standardized_image=Stardadization(Blur_image) --> Facilitate the thresholding
*   Final_image=threshold(Standardized_image) --> If the pixel intensity of the stabdardized image is < threshold , it's replaced by 0 , else we keep the Arcsinh_image values





In [18]:
path_img=path+"/images/"
path_raw=path+"/images/raw/"
path_img_preprocessed2=path+"/images/img_processing_2/"
path_img_preprocessed2_marker=path_img_preprocessed2+"Marker/"
path_img_preprocessed2_biopsies=path_img_preprocessed2+"Biopsies/"

In [19]:
if os.path.isdir(path_img_preprocessed2)==False:
  os.mkdir(path_img_preprocessed2)
  print("✅ Folder for images preprocessed created")
if os.path.isdir(path_img_preprocessed2_biopsies)==False:
  os.mkdir(path_img_preprocessed2_biopsies)
  print("✅ Folder for images preprocessed created")
if os.path.isdir(path_img_preprocessed2_marker)==False:
  os.mkdir(path_img_preprocessed2_marker)

✅ Folder for images preprocessed created
✅ Folder for images preprocessed created


## Functions

In [20]:
# Image preprocessing with arcsinb transformation
def arcsinh_std_thresh(img,thresh,cofactor,kernel):
  img_sin=np.arcsinh(img*cofactor)
  img_med=cv2.medianBlur(img_sin,5)
  img_std=(img_med-np.mean(img_med))/np.std(img_med)
  img_thresh=np.where(img_std<thresh,0,img_sin)
  return img_thresh


## Execution


In [21]:
cofactor=int(input("Enter a cofactor for the arcsinh transformation (100 by default): "))
threshold=int(input("Enter a threshold (2 by default): "))
kernel=int(input("Enter a kernel size (3 by default): "))


Enter a cofactor for the arcsinh transformation (100 by default): 100
Enter a threshold (2 by default): 2
Enter a kernel size (3 by default): 3


In [22]:
total_images = 0
for roi in os.listdir(path_raw):
    roi_dir = os.path.join(path_raw, roi)
    if not os.path.isdir(roi_dir):
        continue
    for marker in os.listdir(roi_dir):
        if os.path.isfile(os.path.join(roi_dir, marker)):
            total_images += 1

with tqdm(total=total_images, desc="IMC preprocessing arcsinh", unit="img", ncols=100) as pbar:
    for roi in os.listdir(path_raw):

        roi_in = os.path.join(path_raw, roi)
        if not os.path.isdir(roi_in):
            continue

        out_biopsy_dir = os.path.join(path_img_preprocessed2_biopsies, roi)
        os.makedirs(out_biopsy_dir, exist_ok=True)

        for marker in os.listdir(roi_in):
            marker_path = os.path.join(roi_in, marker)
            if not os.path.isfile(marker_path):
                continue

            marker_name = marker[:-5]
            out_marker_dir = os.path.join(path_img_preprocessed2_marker, marker_name)
            os.makedirs(out_marker_dir, exist_ok=True)

            img = tiff.imread(marker_path)

            img_proc = arcsinh_std_thresh(img, threshold, cofactor, kernel)

            cv2.imwrite(
                os.path.join(out_biopsy_dir, marker[:-5] + ".png"),
                normalize_255(img_proc)
            )
            cv2.imwrite(
                os.path.join(out_marker_dir, roi + ".png"),
                normalize_255(img_proc)
            )

            pbar.update(1)
print("\n ✅ Processed images ordered by biopsies available there: "+path_img_preprocessed2_biopsies)
print("✅ Processed images ordered by marker available there: "+path_img_preprocessed2_marker)
print(DONE_ASCII)

IMC preprocessing arcsinh:   1%|▎                                  | 8/960 [00:00<01:36,  9.86img/s]/tmp/ipython-input-3233570879.py:5: RuntimeWarning: invalid value encountered in divide
  img=((img-np.min(img))/(np.max(img)-np.min(img)))*255
IMC preprocessing arcsinh: 100%|█████████████████████████████████| 960/960 [01:34<00:00, 10.20img/s]


 ✅ Processed images ordered by biopsies available there: /content/gdrive/MyDrive/these/pipeline/VAA//images/img_processing_2/Biopsies/
✅ Processed images ordered by marker available there: /content/gdrive/MyDrive/these/pipeline/VAA//images/img_processing_2/Marker/

 ____   ___  _   _  _____
|  _ \ / _ \| \ | || ____|
| | | | | | |  \| ||  _|
| |_| | |_| | |\  || |___
|____/ \___/|_| \_||_____|



# Combination of 3 markers
Create a folder with the images, displaying 3 markers of different colors chosen for each biopsy.

In [23]:
path_img=path+"/images/"
path_raw=path+"/images/raw/"
path_img_preprocessed2=path+"/images/img_processing_2/"
path_img_preprocessed2_marker=path_img_preprocessed2+"Marker/"
path_img_preprocessed2_biopsies=path_img_preprocessed2+"Biopsies/"
path_img_preprocessed1=path+"/images/img_processing_1/"
path_img_preprocessed1_marker=path_img_preprocessed1+"Marker/"
path_img_preprocessed1_biopsies=path_img_preprocessed1+"Biopsies/"

##### Functions

In [24]:
def make_rgb(r=None, g=None, b=None):
    """
    r, g, b: 2D arrays already transformed to [0,1]
    returns an RGB image as a float array in [0,1]
    """
    zeros = None
    for c in (r, g, b):
        if c is not None:
            zeros = np.zeros_like(c)
            break
    if zeros is None:
        raise ValueError("At least one channel must be provided.")

    r = r if r is not None else zeros
    g = g if g is not None else zeros
    b = b if b is not None else zeros

    rgb=np.zeros((r.shape[0],r.shape[1],3))
    rgb[:,:,0]=normalize(r)
    rgb[:,:,1]=normalize(g)
    rgb[:,:,2]=normalize(b)
    return rgb


In [25]:
from PIL import ImageDraw, ImageFont # Add missing imports

def annotate_rgb(rgb_float01, labels=None, font_path=None, font_size=15000,
                 margin=10, swatch_size=(30, 30), line_spacing=6, background_alpha=140):
    """
    Adds a colored legend to the image.
    - rgb_float01: RGB float image [0,1] (from make_rgb)
    - labels: dict {"R": "CD16", "G": "CD31", "B": "CD68"} (set to None or omit missing channels)
    - font_path: path to a .ttf font file (optional). If None, Pillow will use the default font.
    - font_size: text size
    - margin: inner margin (px) for the legend box
    - swatch_size: (w,h) of the color square
    - line_spacing: vertical spacing between lines (px)
    - background_alpha: opacity (0–255) of the legend box background
    Returns an annotated RGB np.ndarray (uint8).
    """
    if labels is None:
        labels = {}
    # Convert to 8-bit for PIL
    rgb_u8 = util.img_as_ubyte(np.clip(rgb_float01, 0, 1))
    # Removed 'mode="RGB"' as it's deprecated
    im = Image.fromarray(rgb_u8).convert("RGBA")
    draw_layer = Image.new("RGBA", im.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(draw_layer)

    # Font
    try:
        font = ImageFont.truetype(font_path,font_size) if font_path else ImageFont.load_default()
    except Exception:
        font = ImageFont.load_default()

    # Legend lines from provided labels
    legend_items = []
    if "R" in labels and labels["R"]:
        legend_items.append(((255, 0, 0), labels["R"]))
    if "G" in labels and labels["G"]:
        legend_items.append(((0, 255, 0), labels["G"]))
    if "B" in labels and labels["B"]:
        legend_items.append(((100, 100, 255), labels["B"]))

    if legend_items:
        # Compute legend box dimensions
        sw_w, sw_h = swatch_size
        text_w_max = 0
        text_h = 0
        # Measure text sizes
        for _, txt in legend_items:
            tw, th = draw.textbbox((0, 0), txt, font=font)[2:]
            text_w_max = max(text_w_max, tw)
            text_h = max(text_h, th)

        line_h = max(sw_h, text_h)
        box_w = margin * 3 + sw_w + text_w_max
        box_h = margin * 2 + len(legend_items) * line_h + (len(legend_items) - 1) * line_spacing

        # Position in top-left corner
        x0, y0 = margin, margin
        # Semi-transparent background for readability
        draw.rectangle([x0, y0, x0 + box_w, y0 + box_h], fill=(0, 0, 0, background_alpha))

        # Draw each legend item
        y = y0 + margin
        for color, txt in legend_items:
            # color square
            draw.rectangle([x0 + margin, y, x0 + margin + sw_w, y + sw_h], fill=(*color, 255))
            # text (vertically centered on the line)
            tx = x0 + margin * 2 + sw_w
            ty = y + (line_h - text_h) // 2
            draw.text((tx, ty), txt, font=font, fill=(*color, 255))
            y += line_h + line_spacing

        # Merge drawing layer onto the image
        im = Image.alpha_composite(im, draw_layer)

    return np.array(im.convert("RGB"), dtype=np.uint8)

def generate_rgb_images(path_raw, path_img, list_marker):
    """
    Generates RGB images from three given markers.

    Parameters
    ----------
    path_raw : str
        Path to the folder containing ROIs and their individual channel images (.png).
    path_img : str
        Path to the folder where the generated RGB images will be saved.
    list_marker : list or tuple of str
        List containing exactly three marker names (in the order R, G, B).

    Output
    -------
    Saves annotated RGB images in a subfolder of `path_img`.
    """

    # --- Basic checks ---
    if len(list_marker) != 3:
        raise ValueError("`list_marker` must contain exactly three elements: [R, G, B].")

    # Create output folder
    path_img_rgb = os.path.join(path_img, f"{list_marker[0]}_{list_marker[1]}_{list_marker[2]}")
    os.makedirs(path_img_rgb, exist_ok=True)

    # List ROIs
    rois = sorted([
        d for d in os.listdir(path_raw)
        if os.path.isdir(os.path.join(path_raw, d))
    ])

    # --- Main loop ---
    for roi in tqdm(rois, desc=f"Generating RGB ({list_marker[0]}-{list_marker[1]}-{list_marker[2]})", unit="ROI"):
        roi_path = os.path.join(path_raw, roi)

        # Read channels
        img_r = io.imread(os.path.join(roi_path, f"{list_marker[0]}.png"))
        img_g = io.imread(os.path.join(roi_path, f"{list_marker[1]}.png"))
        img_b = io.imread(os.path.join(roi_path, f"{list_marker[2]}.png"))


        # RGB composition + annotation
        rgb = make_rgb(r=img_r, g=img_g, b=img_b)
        annotated = annotate_rgb(
            rgb,
            labels={"R": list_marker[0], "G": list_marker[1], "B": list_marker[2]},
            font_size=100000
        )

        # Save the image
        out_path = os.path.join(path_img_rgb, f"{roi}.png")
        io.imsave(out_path, util.img_as_ubyte(annotated))

    print(f"\n✅ Generation completed: {len(rois)} ROI(s) processed.")
    print("✅ All the images are available in the folder: " + path_img_rgb)

    print(DONE_ASCII)

##### Display

In [27]:
list_marker=input("Enter the 3 markers separated by a comma: ").split(",")
preprocessing=input("Enter 1 or 2 to choose the preprocessing: ")
if preprocessing=="1":
  preprocessed=path_img_preprocessed1_biopsies
else:
  preprocessed=path_img_preprocessed2_biopsies
generate_rgb_images(preprocessed, path_img, list_marker)


KeyboardInterrupt: Interrupted by user